In [4]:
import pandas as pd
from RoutePlanner.Optimisation import TravelTime
from RoutePlanner import Plot
from RoutePlanner.TemporalCellGrid import TemporalCellGrid
import pickle

In [5]:
OptInfo = {}

# Mesh Information
OptInfo['Mesh']  = {}
OptInfo['Mesh']['Longitude Bounds (Min,Max,Width)']      = [-130,30,5]
OptInfo['Mesh']['Latitude Bounds (Min,Max,Width)']       = [-80,-40,2.5]

OptInfo['Mesh']['Date Range (Min,Max,dT)']               = ['2013-01-1','2013-01-30',1.0]
OptInfo['Mesh']['Homogenous Params (Threshold,Min,Max)'] = [0.25,0.05,0.95] #[0.04,0.05,0.85] - Slow Vehicle, [0.12,0.05,0.85] - SDA
OptInfo['Mesh']['Current Data Path']                     = "../../Data/SOSE_surface_velocity_6yearMean_2005-2010.nc"
OptInfo['Mesh']['Ice Data Path']                         = "../../Data/bsose_i122_2013to2017_1day_SeaIceArea.nc"

# Route Information
OptInfo['Route'] = {} 
OptInfo['Route']['WayPoints']            = '../../resources/WayPoints_org.csv'
OptInfo['Route']['MaxIceExtent']         = 0.8
OptInfo['Route']['Zero Currents']        = True
OptInfo['Route']['VariableSpeed']        = False
OptInfo['Route']['Time Unit']            = 'days'

OptInfo['Route']['VehicleInfo']                = {}
OptInfo['Route']['VehicleInfo']['Speed']       = 26.5
OptInfo['Route']['VehicleInfo']['Unit']        = 'km/hr'
OptInfo['Route']['VehicleInfo']['Beam']        = 24.0
OptInfo['Route']['VehicleInfo']['HullType']    = 'slender'
OptInfo['Route']['VehicleInfo']['ForceLimit']  = 96634.5


In [6]:
# # Generating Cell Grid
# temporalCellGrid = TemporalCellGrid(OptInfo)
# cellGrid         = temporalCellGrid.range(OptInfo['Mesh']['Date Range (Min,Max,dT)'][0],OptInfo['Mesh']['Date Range (Min,Max,dT)'][1],j_grid=True)
# cellGrid.iterativeSplit(0)
# with open('cellGrid.p', 'wb') as f:
#     pickle.dump(cellGrid,f)

with open('cellGrid.p', 'rb') as f:
    cellGrid = pickle.load(f)

In [7]:
# Difference - Land Mesh
cellGrid.cellBoxes[206].landLocked = True

In [9]:
#cellGrid.plot()

map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map)
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.LayerControl(map,collapsed=True)
map

/Users/jsmith/miniconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/jsmith/Documents/Research/Researcher_BAS/RoutePlanning/SDADT-pyRoutePlanner/RoutePlanner/IO.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Polygons['Land'][np.isnan(Polygons['Ice Area'])] = True


In [189]:
TT = TravelTime(cellGrid,OptInfo)
Paths = TT.Paths(source_waypoints=['LongPathEnd'],end_waypoints=['LongPathStart'],verbose=True,multiprocessing=False)
#Paths = TT.Paths(verbose=True,multiprocessing=False)
with open('Paths.p', 'wb') as f:
    pickle.dump(TT,f)

# with open('Paths.p', 'rb') as f:
#     TT = pickle.load(f)

Zero Currets True
=== Processing Waypoint = LongPathEnd ===


In [190]:
map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.MapPaths(Paths,map,PathPoints=False)
map = Plot.LayerControl(map,collapsed=True)
map

/Users/jsmith/miniconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/jsmith/Documents/Research/Researcher_BAS/RoutePlanning/SDADT-pyRoutePlanner/RoutePlanner/IO.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Polygons['Land'][np.isnan(Polygons['Ice Area'])] = True


In [269]:
import numpy as np
from shapely.geometry import Polygon

class _Euclidean_distance():
    """
    Replicating original route planner Euclidean distance 
    Inputs:
      origin      - tuple of floats e.g. (Long_orig,Lat_orig)
      destination - tuple of floats e.g. (Long_dest,Lat_dest)
      Optional: forward - Boolean True or False
    Output:
      Value - If 'forward' is True then returns Distance between 
              two points in 'km'. If 'False' then return the 
              Lat/Long position of a point.

    """

    def __init__(self,scaleLongitude=None):

        self.m_per_latitude  = 111.386*1000.

        if type(scaleLongitude) != type(None):
            self.m_per_longitude = 111.321*1000*np.cos(scaleLongitude*(np.pi/180))
        else:
            self.m_per_longitude = (111.321*1000.)

    def value(self,origin,dest_dist,forward=True):
        lon1,lat1 = origin
        if forward:
            lon2,lat2 = dest_dist
            lon2 = lon2+360
            lon1 = lon1+360
            val = np.sqrt(((lat2-lat1)*self.m_per_latitude)**2 + ((lon2-lon1)*self.m_per_longitude)**2)
        else:
            dist_x,dist_y = dest_dist        
            val = [lon1+(dist_x/self.m_per_longitude),lat1+(dist_y/self.m_per_latitude)]
        return val

class NewtonianCurve:
    def __init__(self,Mesh,DijkstraInfo,OptInfo,unit_shipspeed='km/hr',unit_time='days',debugging=0,maxiter=1000,pathIter=1000,optimizer_tol=1e-3,minimumDiff=1e-6,zerocurrents=True):
        '''
        
        
            BUG:
                - Currently the speed is fixed. Move the construction of the cellBox speed to a function of the cellBox
        
        '''

        # Passing the Mesh information
        self.Mesh = Mesh

        # Passing the Dijkstra Graph
        self.DijkstraInfo = copy.copy(DijkstraInfo)

        # Passing the optional Information
        self.OptInfo = OptInfo


        self.NewtonEpsilon = 0.001
        
        # Inside the code the base units are m/s. Changing the units of the inputs to match
        self.unit_shipspeed = unit_shipspeed
        self.unit_time      = unit_time
        self.s              = self._unit_speed(26.3)
        
        # Information for distance metrics
        self.R              = 6371*1000.
        self.fdist          = _Euclidean_distance()

        # Optimisation Information
        self.maxiter       = maxiter
        self.pathIter      = pathIter
        self.optimizer_tol = optimizer_tol
        self.minimumDiff   = minimumDiff
        self._epsilon      = 1e-4


        # For Debugging purposes 
        self.debugging     = debugging

        # zeroing currents if flagged
        if zerocurrents:
            self.zc = 0.0
        else:
            self.zc = 1.0

    def _unit_speed(self,Val):
        if self.unit_shipspeed == 'km/hr':
            Val = Val*(1000/(60*60))
        if self.unit_shipspeed == 'knots':
            Val = (Val*0.51)
        return Val

    def _unit_time(self,Val):
        if self.unit_time == 'days':
            Val = Val/(60*60*24)
        elif self.unit_time == 'hr':
            Val = Val/(60*60)
        elif self.unit_time == 'min':
            Val = Val/(60)
        elif self.unit_time == 's':
            Val = Val
        return Val





    def _horseshoe(self,case,positive):
        #1. Determine the case of the problem investigating and outline the start and end indices

        #2. List the neighbours for the start and end indices that satisfy that case. 
        #   If they share a common neighbour then a horseshoe cannot be generated & we have an additional case.
        
        #3. From those neighbours, determine their subsequent neighbours determining if any of these lists share a common entry

        #4. Determine the pair that shares the common entry

        #5. Re-inspect the graph adding the crossing points between this pair, adding back in the horseshoe


        if positive:
            sgn = 1
        else:
            sgn = -1

        startGraph = self.DijkstraInfo.loc[self.Sindex]
        endGraph   = self.DijkstraInfo.loc[self.Eindex]

        # Determining only the neighbours across the boundary of interest
        if abs(case)==2:
            startGraphNeighbours = [startGraph['neighbourIndex'][ii] for ii in list(np.where(np.array(startGraph['case'])==sgn*4)[0])]
            endGraphNeighbours   = [endGraph['neighbourIndex'][ii] for ii in list(np.where(np.array(endGraph['case'])==sgn*4)[0])]
        if abs(case)==4:
            startGraphNeighbours = [startGraph['neighbourIndex'][ii] for ii in list(np.where(np.array(startGraph['case'])==sgn*2)[0])]
            endGraphNeighbours   = [endGraph['neighbourIndex'][ii] for ii in list(np.where(np.array(endGraph['case'])==sgn*2)[0])]




        for sGN in startGraphNeighbours:
            for eGN in endGraphNeighbours:
                if (np.array(self.DijkstraInfo.loc[sGN,'neighbourIndex'])==eGN).any() and (np.array(self.DijkstraInfo.loc[eGN,'neighbourIndex'])==sGN).any():
                    sGNGraph = self.DijkstraInfo.loc[sGN]
                    eGNGraph = self.DijkstraInfo.loc[eGN]
                    Crp1 = np.array(startGraph['neighbourCrossingPoints'])[np.where(np.array(startGraph['neighbourIndex']) == sGN)[0][0],:][None,:]
                    Crp2 = np.array(sGNGraph['neighbourCrossingPoints'])[np.where(np.array(sGNGraph['neighbourIndex']) == eGN)[0][0],:][None,:]
                    Crp3 = np.array(eGNGraph['neighbourCrossingPoints'])[np.where(np.array(eGNGraph['neighbourIndex']) == self.Eindex)[0][0],:][None,:]
                    CrossingPoints = np.concatenate((Crp1,Crp2,Crp3))
                    Indices        = np.array([self.Sindex,sGN,eGN,self.Eindex])

                    return CrossingPoints,Indices


        print('Issue Horseshoe - {},{}'.format(sGN,eGN))
        # If is not a horseshoe case but v-shape feature
        CrossingPoints      = np.array([np.nan,np.nan])[None,:]
        Indices             = np.array([np.nan,np.nan])
        return CrossingPoints,Indices        


    def _long_case(self):
            def NewtonOptimisationLong(f,df,y0,x,a,Y,u1,v1,u2,v2,s,R,λ_s,φ_r):
                    tryNum=1
                    iter=0
                    if self.maxiter > 0:
                        while iter < self.maxiter:
                            # -- Maria Original
                            if (iter>100) and (tryNum==1):
                                y0 = (Y*x)/(x+a)
                                tryNum+=1
                            if (iter>200) and (tryNum>=2) and (tryNum <10):
                                tryNum+=1
                                iter-=100
                                if (Y<0):
                                    if v2>v1:
                                        y0 = (tryNum-2)*Y
                                    else:
                                        y0 = (tryNum-3)*-Y
                                else:
                                    if v2<v1:
                                        y0 = (tryNum-2)*Y
                                    else:
                                        y0 = (tryNum-3)*-Y        

                            F,X1,X2  = f(y0,x,a,Y,u1,v1,u2,v2,s,R,λ_s,φ_r)
                            dF = df(y0,x,a,Y,u1,v1,u2,v2,s,R,λ_s,φ_r)
                            imprving = (abs(dF)>1) or ((abs(dF) > self._epsilon*(X1*X2)) and (abs(dF)/iter) > self._epsilon)
                            if not imprving:
                                break
                            y0  -= (F/dF)
                            iter+=1
                    return y0

            def _F(y,x,a,Y,u1,v1,u2,v2,s,R,λ_s,φ_r):
                θ  = (y/R + λ_s)*(np.pi/180)
                zl = x*np.cos(θ)
                ψ  = (-(Y-y)/R + φ_r)*(np.pi/180)
                zr = a*np.cos(ψ)

                C1  = s**2 - u1**2 - v1**2
                C2  = s**2 - u2**2 - v2**2
                D1  = zl*u1 + y*v1
                D2  = zr*u2 + (Y-y)*v2
                X1  = np.sqrt(D1**2 + C1*(zl**2 + y**2))
                X2  = np.sqrt(D2**2 + C2*(zr**2 + (Y-y)**2))

                dzr = -zr*np.sin(ψ)/R
                dzl = -zl*np.sin(θ)/R

                zr_term = (zr - (X2 - D2)*u2/C2)
                zl_term = (zl - (X1 - D1)*u1/C1)
                F = (X1+X2)*y - v1*(X1-D1)*X2/C1 - (Y - v2*(X2-D2)/C2)*X1 + dzr*zr_term*X1 + dzl*zl_term*X2
                return F,X1,X2

            def _dF(y,x,a,Y,u1,v1,u2,v2,s,R,λ_s,φ_r):
                θ  = (y/R +  λ_s)*(np.pi/180)
                zl = x*np.cos(θ)
                ψ  = (-(Y-y)/R + φ_r)*(np.pi/180)
                zr = a*np.cos(ψ)

                C1  = s**2 - u1**2 - v1**2
                C2  = s**2 - u2**2 - v2**2
                D1  = zl*u1 + y*v1
                D2  = zr*u2 + (Y-y)*v2
                X1  = np.sqrt(D1**2 + C1*(zl**2 + y**2))
                X2  = np.sqrt(D2**2 + C2*(zr**2 + (Y-y)**2))

                dzr = -zr*np.sin(ψ)/R
                dzl = -zl*np.sin(θ)/R

                dD1 = dzl*u1 + v1
                dD2 = dzr*u2 - v2
                dX1 = (D1*v1 + C1*y + dzl*(D1*u1 + C1*zl))/X1
                dX2 = (-v2*D2 - C2*(Y-y) + dzr*(D2*u2 + C2*zr))/X2        

                zr_term = (zr - (X2 - D2)*u2/C2)
                zl_term = (zl - (X1 - D1)*u1/C1)

                dF = (X1+X2) + y*(dX1 + dX2) - (v1/C1)*(dX2*(X1-D1) + X2*(dX1-dD1))\
                    + (v2/C2)*(dX1*(X2-D2) + X1*(dX2-dD2))\
                    - Y*dX1 - (zr/(R**2))*zr_term*X1\
                    - (zl/(R**2))*zl_term*X2\
                    + dzr*(dzr-u2*(dX2-dD2))/C2*X1\
                    + dzl*(dzl-u1*(dX1-dD1))/C1*X2\
                    + dzr*zr_term*dX1 + dzl*zl_term*dX2
                return dF 

            Cp = self.Cp
            if self.case == 2:   
                Sp   = self.Sp
                Np   = self.Ep
                Box1 = self.Mesh.cellBoxes[self.Sindex]
                Box2 = self.Mesh.cellBoxes[self.Eindex]
                sgn  = -1
            else:
                Sp   = self.Ep
                Np   = self.Sp
                Box1 = self.Mesh.cellBoxes[self.Eindex]
                Box2 = self.Mesh.cellBoxes[self.Sindex]
                sgn  = 1

            λ_s  = Sp[1]
            φ_r  = Np[1]

            x           = self.fdist.value(Sp,(Cp[0],Sp[1]))
            a           = self.fdist.value(Np,(Cp[0],Np[1]))

            Y           = np.sign(Np[1]-Sp[1])*self.fdist.value((Np[0],Sp[1]),(Np[0],Np[1]))
            #y0          = Y/2
            y0          = np.sign(Cp[1]-Sp[1])*self.fdist.value((Cp[0],Sp[1]),(Cp[0],Cp[1]))


            u1          = sgn*self.zc*Box1.getuC(); v1 = self.zc*Box1.getvC()
            u2          = sgn*self.zc*Box2.getuC(); v2 = self.zc*Box2.getvC()
            y           = NewtonOptimisationLong(_F,_dF,y0,x,a,Y,u1,v1,u2,v2,self.s,self.R,λ_s,φ_r)

            # Updating the crossing points
            CrossingPoints = np.array(self.fdist.value((Cp[0],Sp[1]),(0.0,y),forward=False))[None,:]
            Indices        = np.array([self.Sindex,self.Eindex])

            # # -- SmoothingIssueCase - Horseshoe Case
            if ((Sp[0]==(Box1.cx-Box1.dcx)) or (Sp[0]==(Box1.cx+Box1.dcx))) and ((Np[0]==(Box2.cy-Box2.dcy)) or (Np[0]==(Box2.cy+Box2.dcy))):
                if CrossingPoints[0,1]<=np.max([Box1.cy-Box1.dcy,Box2.cy-Box2.dcy]):
                    CrossingPoints,Indices = self._horseshoe(2,True)
                if CrossingPoints[0,1]>=np.min([Box1.cy+Box1.dcy,Box2.cy+Box2.dcy]):
                    CrossingPoints,Indices = self._horseshoe(2,False)
        
            return CrossingPoints,Indices

    def _lat_case(self):
        def NewtonOptimisationLat(f,df,y0,x,a,Y,u1,v1,u2,v2,s,R,λ,θ,ψ):
                tryNum=1
                iter=0
                if self.maxiter > 0:
                    while iter < self.maxiter:
                        # -- Maria Original
                        if (iter>100) and (tryNum==1):
                            y0 = (Y*x)/(x+a)
                            tryNum+=1
                        if (iter>200) and (tryNum>=2) and (tryNum <10):
                            tryNum+=1
                            iter-=100
                            if (Y<0):
                                if v2>v1:
                                    y0 = (tryNum-2)*Y
                                else:
                                    y0 = (tryNum-3)*-Y
                            else:
                                if v2<v1:
                                    y0 = (tryNum-2)*Y
                                else:
                                    y0 = (tryNum-3)*-Y        

                        # --- Updating ---
                        F,X1,X2  = f(y0,x,a,Y,u1,v1,u2,v2,s,R,λ,θ,ψ)
                        dF = df(y0,x,a,Y,u1,v1,u2,v2,s,R,λ,θ,ψ)
                        imprving = (abs(dF)>1) or ((abs(dF) > self._epsilon*(X1*X2)) and (abs(dF)/iter) > self._epsilon)
                        if not imprving:
                            break
                        y0 -= (F/dF)

                        iter +=1

                return y0

        def _F(y,x,a,Y,u1,v1,u2,v2,s,R,λ,θ,ψ):
            λ   = λ*(np.pi/180)
            ψ   = ψ*(np.pi/180)
            θ   = θ*(np.pi/180)
            r1  = np.cos(λ)/np.cos(θ)
            r2  = np.cos(ψ)/np.cos(θ)

            d1  = np.sqrt(x**2 + (r1*y)**2)
            d2  = np.sqrt(a**2 + (r2*(Y-y))**2)
            C1  = s**2 - u1**2 - v1**2
            C2  = s**2 - u2**2 - v2**2
            D1  = x*u1 + r1*v1*y
            D2  = a*u2 + r2*v2*(Y-y)
            X1  = np.sqrt(D1**2 + C1*(d1**2))
            X2  = np.sqrt(D2**2 + C2*(d2**2)) 



            F = ((r2**2)*X1 + (r1**2)*X2)*y - ((r1*(X1-D1)*X2*v1)/C1) - r2*(r2*Y-v2*(X2-D2)/C2)*X1

            return F,X1,X2

        def _dF(y,x,a,Y,u1,v1,u2,v2,s,R,λ,θ,ψ):
            λ   = λ*(np.pi/180)
            ψ   = ψ*(np.pi/180)
            θ   = θ*(np.pi/180)
            r1  = np.cos(λ)/np.cos(θ)
            r2  = np.cos(ψ)/np.cos(θ)


            d1  = np.sqrt(x**2 + (r1*y)**2)
            d2  = np.sqrt(a**2 + (r2*(Y-y))**2)
            C1  = s**2 - u1**2 - v1**2
            C2  = s**2 - u2**2 - v2**2
            D1  = x*u1 + r1*v1*y
            D2  = a*u2 + r2*v2*(Y-y)
            X1  = np.sqrt(D1**2 + C1*(d1**2))
            X2  = np.sqrt(D2**2 + C2*(d2**2))   
            
            dX1 = (r1*(D1*v1 + r1*C1*y))/X1
            dX2 = (r2*(D2*v2 - r2*C2*(Y-y)))/X2


            dF = ((r2**2)*X1 + (r1**2)*X2) + y*((r2**2)*dX1 + (r1**2)*dX2)\
                - ((r1*v1)/C1)*((X1-D1)*dX2 + (dX1-r1*v1)*X2)\
                + ((r2*v2)/C2)*((X2-D2)*dX1 + (dX2+r2*v2)*X1)\
                - (r2**2)*Y*dX1
            return dF


        Cx,Cy = self.Cp

        if self.case == -4:   
            Sx,Sy = self.Sp
            Nx,Ny = self.Ep
            Cl1   = self.Mesh.cellBoxes[self.Sindex]
            Cl2   = self.Mesh.cellBoxes[self.Eindex]
            sgn1  = 1
            sgn2  = -1
        else:
            Sx,Sy = self.Ep
            Nx,Ny = self.Sp
            Cl1   = self.Mesh.cellBoxes[self.Eindex]
            Cl2   = self.Mesh.cellBoxes[self.Sindex]  
            sgn1  = -1
            sgn2  = 1


        λ=Sy
        θ=Cy #Cy    
        ψ=Ny #S   

        x     = self.fdist.value((Sx,Sy),(Sx,Cy))
        a     = self.fdist.value((Nx,Ny),(Nx,Cy))
        Y     = np.sign(Nx-Sx)*self.fdist.value((Sx,Sy),(Sx,Ny))
        Su    = sgn1*self.zc*Cl1.getvC(); Sv = sgn2*self.zc*Cl1.getuC()
        Nu    = sgn1*self.zc*Cl2.getvC(); Nv = sgn2*self.zc*Cl2.getuC()
        #y0    = Y/2

        y0    = np.sign(Cx-Sx)*self.fdist.value((Sx,Cy),(Cx,Cy))

        y     = NewtonOptimisationLat(_F,_dF,y0,x,a,Y,Su,Sv,Nu,Nv,self.s,self.R,λ,θ,ψ)

        CrossingPoints  = np.array(self.fdist.value((Sx,Cy),(y,0.0),forward=False))[None,:]
        Indices         = np.array([self.Sindex,self.Eindex])

        # # -- SmoothingIssueCase - Close Distance
        # if abs(Cy-Sy)<1e-4 and abs(Cy-Ny)<1e-4:
        #     CrossingPoints  = np.array([Nx,Cy])[None,:]
        #     Indices         = np.array([self.Sindex])
        #     return CrossingPoints,Indices


        # -- SmoothingIssueCase - Horseshoe Case
        if ((Sy==(Cl1.cy-Cl1.dcy)) or (Sy==(Cl1.cy+Cl1.dcy))) and ((Ny==(Cl2.cy-Cl2.dcy)) or (Ny==(Cl2.cy+Cl2.dcy))):
            if CrossingPoints[0,0]<=np.max([Cl1.cx-Cl1.dcx,Cl2.cx-Cl2.dcx]):
                CrossingPoints,Indices = self._horseshoe(4,False)
            if CrossingPoints[0,0]>=np.min([Cl1.cx+Cl1.dcx,Cl2.cx+Cl2.dcy]):
                CrossingPoints,Indices = self._horseshoe(4,True)

        return CrossingPoints,Indices

    def _corner_case(self):
        '''
        '''
        # Separting out the Long/Lat of each of the points
        Xs,Ys = self.Sp
        Xc,Yc = self.Cp
        Xe,Ye = self.Ep

        # === 1. Assess the cells that are shared commonly in the corner case ====
        sourceNeighbourIndices = self.DijkstraInfo.loc[self.Sindex]
        endNeighbourIndices    = self.DijkstraInfo.loc[self.Eindex]
    
        commonIndices = list(set(sourceNeighbourIndices['neighbourIndex']).intersection(endNeighbourIndices['neighbourIndex']))
        CornerCells   = self.DijkstraInfo.loc[commonIndices]
        Y_line = ((Yc-Ys)/(Xc-Xs))*(Xe-Xs) + Ys

        if (abs(self.case)==1):
            if Ye > Y_line:
                newCell = CornerCells.loc[CornerCells['cY'].idxmax()]
            elif Ye <= Y_line:
                newCell = CornerCells.loc[CornerCells['cY'].idxmin()]
        if (abs(self.case)==3):
            if Ye >= Y_line:
                newCell = CornerCells.loc[CornerCells['cY'].idxmax()]
            elif Ye < Y_line:
                newCell = CornerCells.loc[CornerCells['cY'].idxmin()]

        # === 3. Return the path crossing points and cell indices
        firstCrossingPoint  = np.array(sourceNeighbourIndices['neighbourCrossingPoints'])[np.where(np.array(sourceNeighbourIndices['neighbourIndex'])==newCell.name)[0][0],:]
        secondCrossingPoint = np.array(newCell['neighbourCrossingPoints'])[np.where(np.array(newCell['neighbourIndex'])==endNeighbourIndices.name)[0][0],:]

        
        CrossingPoints      = np.concatenate((firstCrossingPoint[None,:],secondCrossingPoint[None,:]))
        Indices             = np.array([self.Sindex,newCell.name,self.Eindex])
        return CrossingPoints,Indices


    def _updateCrossingPoint(self):
        '''
            BUG:
            --> 
        '''

        # # -- SmoothingIssueCase - Reverse Edge
        if self.Sindex == self.Eindex:
            return None,None



        # -- Updating the crossing points
        if self.debugging>0:
            print('===========================================================')
        if abs(self.case)==2:
            CrossingPoints,Indices = self._long_case()
        elif abs(self.case)==4:
            CrossingPoints,Indices = self._lat_case()
        elif (abs(self.case)==1) or (abs(self.case)==3):
            CrossingPoints,Indices = self._corner_case()

        return CrossingPoints,Indices

    def PathSmoothing(self,path,cellIndices):
        import copy
        self.path        = copy.copy(path)
        self.cellIndices = copy.copy(cellIndices)

        iter = 0
        while iter <= self.pathIter:
            
            print(iter)
            id = 0

            self._current_path = copy.copy(self.path)

            # Updating all points along path
            while id <= (len(self.path) - 3):
                self.Sp     = tuple(self.path[id,:])
                self.Cp     = tuple(self.path[id+1,:])
                self.Ep     = tuple(self.path[id+2,:])

                self.Sindex = self.cellIndices[id]
                self.Eindex = self.cellIndices[id+1]


                self.sourceNeighbourIndices = self.DijkstraInfo.loc[self.Sindex]
                self.endNeighbourIndices    = self.DijkstraInfo.loc[self.Eindex]

                self.case = self.sourceNeighbourIndices['case'][np.where(np.array(self.sourceNeighbourIndices['neighbourIndex'])==self.endNeighbourIndices.name)[0][0]]

                Points,Indices = self._updateCrossingPoint()
                
                # # # # == SmoothingCaseIssue - Reverse Case
                # if (type(Points)==type(None)) and (type(Indices)==type(None)):
                #     self.path        = np.delete(self.path,id,0)
                #     self.cellIndices = np.delete(self.cellIndices,id,0)   
                #     id+=1
                #     continue           
                
                # # ======== SmoothingCaseIssue - NaNs ==========
                # #    Issue: Returned NaN Value
                # # Solution: Don't Update Crossing Point
                # if np.isnan(Points).any():
                #     id+=1
                #     continue

                # # ======== SmoothingCaseIssue - Ice/Land Cell ==========
                # #    Issue: Entered an Inaccessible cell consisting of Land or inaccessible Ice content
                # # Solution: Do not update crossing points as entered a land cell
                # Allowed = True
                # Boxes = [self.Mesh.cellBoxes[i] for i in Indices]
                # for box in Boxes:
                #     if box.containsLand() or box.iceArea() >= self.OptInfo['MaxIceExtent']:
                #         Allowed = False
                # if not Allowed:
                #     id+=1
                #     continue

                # Updating the crossing points
                self.path[id+1,:]      = Points[0,:]
                if Points.shape[0] > 1:
                    self.path        = np.insert(self.path,id+2,Points[1:,:],0)

                # Updating the Indices
                self.cellIndices[id]   = Indices[0]
                self.cellIndices[id+1] = Indices[-1]
                if len(Indices) > 1:
                    self.cellIndices = np.insert(self.cellIndices,id+1,Indices[1:],0)

                id+=1
                #id+=((len(Indices)-2)+1)

            iter+=1

            # Stop optimisation if the points are within some minimum difference
            if iter!=0:
                if self.path.shape == self._current_path.shape:
                    if np.max(np.sqrt((self.path-self._current_path)**2)) < self.minimumDiff:
                        break

In [270]:
# import copy
# SmoothedPaths=[]

# with open('Paths.p', 'rb') as f:
#     TT = pickle.load(f)


# ii=0

# Path = copy.copy(TT.paths[ii])

# startPoint = Path['Path']['Points'][0,:][None,:]
# endPoint   = Path['Path']['Points'][-1,:][None,:]

# Points      = np.concatenate([startPoint,Path['Path']['Points'][1:-1:2],endPoint])
# cellIndices = Path['Path']['CellIndices']

# nc = NewtonianCurve(TT.Mesh,TT.DijkstraInfo[Path['from']],TT.OptInfo,maxiter=0,zerocurrents=True)
# nc.PathSmoothing(Points,cellIndices)

# # Path = copy.copy(TT.paths[ii])
# # Path['Path']['Points']       = nc.path
# # SmoothedPaths.append(Path)

In [279]:
import copy
from matplotlib.patches import Polygon as MatplotPolygon


with open('Paths.p', 'rb') as f:
    TT = pickle.load(f)
ii=0
Path = copy.copy(TT.paths[ii])

nc = NewtonianCurve(TT.Mesh,TT.DijkstraInfo[Path['from']],TT.OptInfo,maxiter=1,zerocurrents=True)

startPoint = Path['Path']['Points'][0,:][None,:]
endPoint   = Path['Path']['Points'][-1,:][None,:]

Points      = np.concatenate([startPoint,Path['Path']['Points'][1:-1:2],endPoint])
cellIndices = Path['Path']['CellIndices']


nc.path        = copy.copy(Points)
nc.cellIndices = copy.copy(cellIndices)



In [280]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MatplotPolygon
iter = 0
nc.pathIter=100

while iter <= nc.pathIter:
    
    print(iter)
    id = 0
    nc._current_path = copy.copy(nc.path)
    while id <= (len(nc.path) - 3):
        nc.Sp     = tuple(nc.path[id,:])
        nc.Cp     = tuple(nc.path[id+1,:])
        nc.Ep     = tuple(nc.path[id+2,:])

        nc.Sindex = nc.cellIndices[id]
        nc.Eindex = nc.cellIndices[id+1]

        # # -- Dropping Points that are at the same location --
        # if np.sqrt((nc.Ep[0]-nc.Cp[0])**2 + (nc.Ep[1]-nc.Cp[1])**2) ==0:
        #     nc.path        = np.delete(nc.path,id+1,0)
        #     nc.cellIndices = np.delete(nc.cellIndices,id+1,0)   
        #     continue           


        nc.sourceNeighbourIndices = nc.DijkstraInfo.loc[nc.Sindex]
        nc.endNeighbourIndices    = nc.DijkstraInfo.loc[nc.Eindex]

        nc.case = nc.sourceNeighbourIndices['case'][np.where(np.array(nc.sourceNeighbourIndices['neighbourIndex'])==nc.endNeighbourIndices.name)[0][0]]

        Points,Indices = nc._updateCrossingPoint()


        # # -- SmoothingSpecialCase - Reverse Edge
        if (type(Points)==type(None)) and (type(Indices)==type(None)):
            nc.path        = np.delete(nc.path,id,0)
            nc.cellIndices = np.delete(nc.cellIndices,id,0)   
            id+=1
            continue           

        # -- Neglect Path Update if Ice or Land
        Allowed = True
        Boxes = [nc.Mesh.cellBoxes[i] for i in Indices]
        for box in Boxes:
            if box.containsLand() or box.iceArea() >= nc.OptInfo['MaxIceExtent']:
                Allowed = False
        if not Allowed:
            id+=1
            continue 


        # Updating the Indices
        nc.cellIndices[id]   = Indices[0]
        nc.cellIndices[id+1] = Indices[-1]
        if len(Indices) > 2:
            nc.cellIndices = np.insert(nc.cellIndices,id+1,Indices[1:-1],0)


        # Updating the crossing points
        nc.path[id+1,:]      = Points[0,:]
        if Points.shape[0] > 1:
            nc.path        = np.insert(nc.path,id+2,Points[1:,:],0)
        else:
            id+=1



        # print(nc.Sindex,nc.Eindex)
        # fig = plt.figure()
        # plt.scatter(nc.Sp[0],nc.Sp[1],50,'r')
        # plt.scatter(nc.Ep[0],nc.Ep[1],50,'r')
        # plt.scatter(nc.Cp[0],nc.Cp[1],50,'b')
        # for indx in Indices:
        #     plt.gca().add_patch(MatplotPolygon(cellGrid.cellBoxes[indx].getBounds(), closed=True, fill=False, color='grey', alpha=1,linewidth=2.0))
        # plt.scatter(Points[:,0],Points[:,1],5,'k')


        # Check if can update further

        #id+=(len(Indices)-2 + 1)

    iter+=1

    # Stop optimisation if the points are within some minimum difference
    if iter!=0:
        if nc.path.shape == nc._current_path.shape:
            if np.max(np.sqrt((nc.path-nc._current_path)**2)) < 1e-8:
                break


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


/var/folders/6y/sjc012x16xvc6m5s18y8jl200000gn/T/ipykernel_1072/545245078.py:234: RuntimeWarning: invalid value encountered in double_scalars
  dX1 = (D1*v1 + C1*y + dzl*(D1*u1 + C1*zl))/X1
/var/folders/6y/sjc012x16xvc6m5s18y8jl200000gn/T/ipykernel_1072/545245078.py:366: RuntimeWarning: invalid value encountered in double_scalars
  dX1 = (r1*(D1*v1 + r1*C1*y))/X1
/var/folders/6y/sjc012x16xvc6m5s18y8jl200000gn/T/ipykernel_1072/545245078.py:235: RuntimeWarning: invalid value encountered in double_scalars
  dX2 = (-v2*D2 - C2*(Y-y) + dzr*(D2*u2 + C2*zr))/X2


20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [281]:
SmoothedPaths = []
Path = copy.copy(TT.paths[ii])
Path['Path']['Points']       = nc.path
SmoothedPaths.append(Path)

In [282]:
import json

import json
import folium
import numpy as np
import copy

with open('LongPathStart.json', 'r') as f:\
    MPaths = np.array(json.load(f)['features'][0]['geometry']['coordinates'])


def MapMPaths(MPaths,map,color='blue',PathPoints=True):
    Pths        = folium.FeatureGroup(name='Paths')
    Pths_points = folium.FeatureGroup(name='Path Points')
    for path in copy.deepcopy(Paths):
        points = MPaths
        # points[:,0] = points[:,0]
        points = points[:,::-1]
        folium.PolyLine(points,color=color, weight=2.0, opacity=1).add_to(Pths)
        for idx in range(len(points)):
            loc = [points[idx,0],points[idx,1]]
            folium.Marker(
                location=loc,
                icon=folium.plugins.BeautifyIcon(icon='circle',
                                            border_color='transparent',
                                            background_color='transparent',
                                            border_width=1,
                                            text_color=color,
                                            inner_icon_style='margin:0px;font-size:0.8em')
            ).add_to(Pths_points)
    Pths.add_to(map)
    if PathPoints:
        Pths_points.add_to(map)
    return map


with open('LPSE2_geo.json', 'r') as f:\
    MPaths = np.array(json.load(f)['features'][0]['geometry']['coordinates'])
A = pd.read_csv('GroundTruth_LongPath_End2Start_Smooth.csv')


def GreatCircle(Start_p,End_p):
    import pyproj
    import numpy as np
    startlong, startlat = Start_p
    endlong, endlat     = End_p

    # calculate distance between points
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(startlong, startlat, endlong, endlat)

    # calculate line string along path with segments <= 1 km
    lonlats = g.npts(startlong, startlat, endlong, endlat,
                    1 + int(dist / 1000))

    lonlats = np.array(lonlats)

    return lonlats




map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = MapMPaths(np.array(A),map,PathPoints=True)
map = MapMPaths(GreatCircle((-129.0,-58.0),(-58.0,-58.0)),map,PathPoints=False,color='red')
map = Plot.MapPaths(SmoothedPaths,map,PathPoints=True)
map = Plot.MapPaths(Paths,map,PathPoints=False)
map = Plot.LayerControl(map,collapsed=True)
map

/Users/jsmith/miniconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/jsmith/Documents/Research/Researcher_BAS/RoutePlanning/SDADT-pyRoutePlanner/RoutePlanner/IO.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Polygons['Land'][np.isnan(Polygons['Ice Area'])] = True


In [ ]:
import folium
from RoutePlanner.CellBox import CellBox
from pyproj import Geod

Path = copy.copy(TT.paths[ii])
Path['Path']['Points']       = nc.path
SmoothedPaths.append(Path)

def MapPaths(Paths,map,PathPoints=True):
    Pths        = folium.FeatureGroup(name='Paths')
    Pths_points = folium.FeatureGroup(name='Path Points')
    for path in copy.deepcopy(Paths):
        points = path['Path']['Points']
        points[:,0] = points[:,0]
        points = points[:,::-1]
        folium.PolyLine(points,color="black", weight=2.0, opacity=1,
                        popup='{}->{}\n Travel-Time = {:.2f}days'.format(path['from'],path['to'],path['Time'])).add_to(Pths)
        for idx in range(len(points)):
            loc = [points[idx,0],points[idx,1]]
            folium.Marker(
                location=loc,
                icon=folium.plugins.BeautifyIcon(icon='circle',
                                            border_color='transparent',
                                            background_color='transparent',
                                            border_width=1,
                                            text_color='black',
                                            inner_icon_style='margin:0px;font-size:0.6em')
            ).add_to(Pths_points)
    Pths.add_to(map)
    if PathPoints:
        Pths_points.add_to(map)
    return map


def MapCurrents(cellGrid,map,show=False,scale=15):
    import folium
    from pyproj import Geod
    def bearing(st,en):
        import numpy as np
        long1,lat1 = st
        long2,lat2 = en
        dlong = long2-long1
        dlat  = lat2-lat1
        vector_1 = [0, 1]
        vector_2 = [dlong, dlat]
        if np.linalg.norm(vector_2) == 0:
            return np.nan
        unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
        unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
        dot_product = np.dot(unit_vector_1, unit_vector_2)
        angle = np.arccos(dot_product)/(np.pi/180)*np.sign(vector_2[0])
        if (angle==0) & (np.sign(dlat)==-1):
            angle=180
        if angle < 0:
            angle = angle +360
        angle
        return angle

    cellGrid
    X=[];Y=[];U=[];V=[];
    for ii in range(len(cellGrid.cellBoxes)):
        cellbox = cellGrid.cellBoxes[ii]
        if not isinstance(cellbox, CellBox):
            continue

        X.append(cellbox.cx)
        Y.append(cellbox.cy)
        U.append(cellbox.getuC())
        V.append(cellbox.getvC())
    Currents = pd.DataFrame({'X':X,'Y':Y,'U':U,'V':V})
    Currents = Currents.dropna()


    vectors = folium.FeatureGroup(name='Currents',show=show)
    for idx,vec in Currents.iterrows():
        loc =[[vec['Y'],vec['X']],[vec['Y']+vec['V']*scale,vec['X']+vec['U']*scale]]
        folium.PolyLine(loc, color="black").add_to(vectors)
        # get pieces of the line
        pairs = [(loc[idx], loc[idx-1]) for idx, val in enumerate(loc) if idx != 0]
        # get rotations from forward azimuth of the line pieces and add an offset of 90°
        geodesic = Geod(ellps='WGS84')
        rotations = [geodesic.inv(pair[0][1], pair[0][0], pair[1][1], pair[1][0])[0]+90 for pair in pairs]
        # create your arrow
        for pair, rot in zip(pairs, rotations):
            folium.RegularPolygonMarker(location=pair[0], color='black', fill=True, fill_color='black', fill_opacity=1,
                                        number_of_sides=3, rotation=rot,radius=2).add_to(vectors)

    vectors.add_to(map)
    return map




map = Plot.BaseMap(logo=False)
map = Plot.MapMesh(cellGrid,map,threshold=OptInfo['Route']['MaxIceExtent'])
map = Plot.MapWaypoints(pd.read_csv('../../resources/WayPoints.csv'),map)
map = MapPaths(SmoothedPaths,map,PathPoints=True)
#map = MapCurrents(cellGrid,map,show=True,scale=1)
#map = Plot.TimeMapPaths(Paths,map,starttime=OptInfo['Start Time'])
map = Plot.LayerControl(map,collapsed=True)
map

In [ ]:
map = Plot.BaseMap(logo=False)
map = MapCurrents(cellGrid,map,show=True,scale=50)
map

In [ ]:
np.array(nc.sourceNeighbourIndices['neighbourIndex'])==nc.endNeighbourIndices.name

In [ ]:
nc.sourceNeighbourIndices['case'][np.where(np.array(nc.sourceNeighbourIndices['neighbourIndex'])==nc.endNeighbourIndices.name)[0][0]]

In [ ]:
import folium
from pyproj import Geod
def bearing(st,en):
    import numpy as np
    long1,lat1 = st
    long2,lat2 = en
    dlong = long2-long1
    dlat  = lat2-lat1
    vector_1 = [0, 1]
    vector_2 = [dlong, dlat]
    if np.linalg.norm(vector_2) == 0:
        return np.nan
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)/(np.pi/180)*np.sign(vector_2[0])
    if (angle==0) & (np.sign(dlat)==-1):
        angle=180
    if angle < 0:
        angle = angle +360
    angle
    return angle

cellGrid
X=[];Y=[];U=[];V=[];
for ii in range(len(cellGrid.cellBoxes)):
    cellbox = cellGrid.cellBoxes[ii]
    if not isinstance(cellbox, CellBox):
        continue

    X.append(cellbox.cx)
    Y.append(cellbox.cy)
    U.append(cellbox.getuC())
    V.append(cellbox.getvC())
Currents = pd.DataFrame({'X':X,'Y':Y,'U':U,'V':V})
Currents = Currents.dropna()
Currents['X'] = Currents['X'] - 360

In [ ]:
Currents

In [143]:
nc.cellIndices

array([140, 172, 204, 205, 237, 238, 270, 271, 303, 302, 334, 333, 365,
       366, 398, 399, 431, 430, 462])